In [1]:
from jsonql import smart_open, read_jsons
import os
import matplotlib.pyplot as plt
import perplexity
from pathlib import Path
from sentence_splitter import SentenceSplitter, split_text_into_sentences
import re
import json
from tqdm import tqdm
from multiprocessing import Pool
import itertools
import numpy as np
import subprocess
import pandas as pd

In [2]:
SPLITS = ['head', 'middle', 'tail']

langs_splitter = (['ca' ,'cs', 'da', 'nl', 'en', 'fi', 'fr', 'de']
                + ['el', 'hu', 'is', 'it', 'lv' ,'lt', 'no', 'pl']
                + ['pt', 'ro', 'ru', 'sk', 'sl', 'es' ,'sv', 'tr']
                 )
DEVANHAGHRI_LANGUAGES = ['as', 'bh', 'bn', 'bpy', 'dty', 'hi', 'ks', 'mai', 'ne']

DEVANHAGHRI_LANGUAGES_WITH_DOT = ['gom', 'gu', 'mr']

CYRILIC_LANGUAGES = [
    "ab", "ady", "ba", "be", "be_x_old", "bg", "bxr", "ce", "cu", "cv",
    "inh", "kbd", "kk", "koi", "kv", "ky", "lbe", "lez", "mdf", "mhr",
    "mk", "mn", "tt", "tyv", "udm", "uk", "xal" ]

ARABIC_LANGUAGES = ['arc', 'arz', 'azb', 'fa', 'glk', 'lrc', 'ug', 'ur', 'ar']

def split_chinese(paragraph):
    return re.findall(u'[^!?。\.\!\?]+[!?。\.\!\?]?', paragraph, flags=re.U)

def SplitDevanhaghri(line):
    return re.findall(u'[^!?।৷\!\?]+[!?।৷\?\!]?', line, flags=re.U)

# same above but latin point is also an EOS marker
def SplitDevanhaghriPoint(line):
    return re.findall(u'[^!?।৷\.\!\?]+[!?।৷\.\?\!]?', line, flags=re.U)

In [68]:
folder = "/checkpoint/guw/cc_clean2/regroup/2019-09/"

# get file sizes, langs

In [36]:
command = "ls  /checkpoint/guw/cc_clean2/regroup/2019-09/*_0000.json.gz | xargs -n1 -P200  -I {} bash -c 'echo \"{} -> $(zcat {} |  wc -l)\"'"
p = subprocess.Popen(command , stdout=subprocess.PIPE, shell=True)
ls_res = p.communicate()[0].decode("utf-8").split('\n')

In [6]:
#command2 = "ls  /checkpoint/guw/cc_clean2/regroup/2019-13/*_0000.json.tmp.gz | xargs -n1 -P200  -I {} bash -c 'echo \"{} -> $(zcat {} |  wc -l)\"'"
#p2 = subprocess.Popen(command2 , stdout=subprocess.PIPE, shell=True)
#ls_res2 = p2.communicate()[0].decode("utf-8").split('\n')

In [37]:
nb_doc_tmp = {}
#for data in ls_res + ls_res2:
for data in ls_res :
    if len(data) == 0:
        continue
    nb_doc_tmp['_'.join(data.split('/')[-1].split('_')[0:2])] = int(data.split(' ')[-1])

In [69]:
sizes = {}
langs = {}
for file in sorted(os.listdir(folder)):
    if file.endswith('index') or file == 'log':
        continue
    name = '_'.join(file.split('_')[0:2])
    if name in sizes:
        sizes[name] += os.path.getsize(folder + file)
    else:
        sizes[name] = os.path.getsize(folder + file)
    lang = file.split('_')[0]
    if lang not in langs :
        langs[lang] = []
    split = name.split('_')[1]
    if not split in langs[lang]:
        langs[lang].append(split)
normalizer = {}

for name,size in sizes.items(): 
    file = name + '_0000.json.gz' 
    assert os.path.exists(folder + file)
    normalizer[name] = os.path.getsize(folder + file) / size 


# compute number of words

In [ ]:
def get_prefixs(langs):
    for key, values in langs.items():
        for value in values:
            yield (key,value)

In [ ]:
def get_counts(inputs):
    # compute number of words for each lang each split
    assert len(inputs) == 2
    lang = inputs[0]
    part = inputs[1]
    prefix = lang + '_' + part
    
    mose_tokenizer = perplexity.RobustTokenizer(lang)
    if lang in langs_splitter :
        splitter = SentenceSplitter(language=lang)
    elif lang in CYRILIC_LANGUAGES:
        splitter = SentenceSplitter(language='ru')
    elif lang in ARABIC_LANGUAGES:
        splitter = SentenceSplitter(language='en')
    
    file = prefix + '_0000.json.gz' 
    assert os.path.exists(folder + file)
    source = smart_open(folder + file)
    docs = read_jsons(source)
    count_words = 0 
    count_sentences = 0 
    
    for i,doc in enumerate(docs):
        if i > 100000:
            break
        for paragraph in doc['raw_content'].split('\n'):
            text_tokenized = mose_tokenizer.do(paragraph)
            count_words += len(text_tokenized.split(' '))
            if lang in langs_splitter or lang in CYRILIC_LANGUAGES:
                text_splitted = splitter.split(text_tokenized)
                count_sentences += len(text_splitted)
            elif lang in ARABIC_LANGUAGES:
                text_tokenized = re.sub("؟", "?", text_tokenized)
                text_splitted = splitter.split(text_tokenized)
                count_sentences += len(text_splitted)
            elif lang in DEVANHAGHRI_LANGUAGES:
                text_splitted = SplitDevanhaghri(text_tokenized)
                count_sentences += len(text_splitted)
            elif lang in DEVANHAGHRI_LANGUAGES_WITH_DOT:
                text_splitted = SplitDevanhaghriPoint(text_tokenized)
                count_sentences += len(text_splitted)
            elif lang == 'zh' or 'ja':
                text_splitted = split_chinese(text_tokenized)
                count_sentences += len(text_splitted)       
    return (prefix, count_words,count_sentences, i)

In [ ]:
def get_ratios_after_dedup(inputs):
    # compute number of words for each lang each split
    assert len(inputs) == 2
    lang = inputs[0]
    part = inputs[1]
    prefix = lang + '_' + part
    print("opening " + prefix)
    file = prefix + '_0000.json.gz' 
    assert os.path.exists(folder + file)
    source = smart_open(folder + file)
    docs = read_jsons(source)
    original_length = 0 
    final_length = 0 
    for i,doc in enumerate(docs):
        original_length += doc['original_length']
        final_length += doc['length']
    ratio = final_length/original_length
    print("closing" + prefix)
    return prefix, ratio

In [ ]:
# prefixs = get_prefixs(langs)
# pool = Pool()
# print("start ratios_after_dedup ...")
# results = pool.map(get_ratios_after_dedup,prefixs)
# print("end ratios_after_dedup")

# ratios_tmp = {}
# for result in results:
#     prefix = result[0]
#     ratio = result[1]
#     ratios_tmp[prefix] = ratio

In [20]:
# compute number of words for each lang each split
prefixs = get_prefixs(langs)
pool = Pool()
print("start get_counts ...")
results = pool.map(get_counts,prefixs)
print("end get_counts")


nb_words_tmp = {}
nb_sentences_tmp = {}

for result in results:
    prefix = result[0]
    nb_w = result[1]
    nb_s = result[2]
    i = result[3] + 1
    nb_words_tmp[prefix] = int( nb_w * nb_doc_tmp[prefix] / i / normalizer[prefix] )
    nb_sentences_tmp[prefix] = int( nb_s  * nb_doc_tmp[prefix] / i / normalizer[prefix] )

In [39]:
# estimate number of words for whole dump
nb_words = { lang:0 for lang in langs }
nb_sentences = { lang:0 for lang in langs }
nb_docs  = {lang:0 for lang in langs}
ratios = {lang:0 for lang in langs}
for lang in langs:
    total_size = 0
    for split in langs[lang]:
        prefix = lang + '_' + split
        nb_words[lang] += nb_words_tmp[prefix]
        nb_sentences[lang] += nb_sentences_tmp[prefix]
        nb_docs[lang] += int(nb_doc_tmp[prefix] / normalizer[prefix])
        ratios[lang] += ratios_tmp[prefix] * sizes[prefix]
        total_size += sizes[prefix]
    ratios[lang] /= total_size
    ratios[lang] = "%.2f" % ratios[lang]
    nb_sentences[lang] = str(nb_sentences[lang]) if nb_sentences[lang] > 0 else '-'

In [60]:
size_by_langs = {}
for lang,splits in langs.items():
    size_by_langs[lang] = 0
    for split in splits:
        size_by_langs[lang] += sizes[lang+'_'+split]

In [40]:
df = pd.DataFrame({'language': list(langs.keys()), 
                    'number documents': list(nb_docs.values()),
                    'number sentences': list(nb_sentences.values()),
                    'number tokens': list(nb_words.values()),
                    'size in bytes': list(size_by_lang.values())
#                     'percentage after dedup': list(ratios.values())
                  }, 
                  columns= ['language',
                            'number documents',
                            'number sentences',
                            'number tokens', 
                            'size in bytes']
                           # 'percentage after dedup']
                 )
export_csv = df.to_csv('/private/home/malachaux/ccall/stats_2019-09_all.csv', index = None, header=True)
print('stats saved in stats_2019_09_all.csv ')

stats saved in stats.csv 
